In [1]:
################# ALL THE IMPROTS ###################
import os
import PIL
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Sequential

from keras.layers.reshaping.reshape import Reshape

In [2]:
################################################################################
############################## IMAGE GENERATOR #################################

img_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [3]:
################################################################################
########################### FLOW FROM DIRECTORY#################################

face_train = img_generator.flow_from_directory('data/train_images',target_size=(512,512),shuffle=False,
                                               seed=40,save_format='jpg', batch_size=16, classes=None,
                                               class_mode=None)

comic_train = img_generator.flow_from_directory('data/train_cartoon',target_size=(512,512),shuffle=False,
                                                seed=40,save_format='jpg',batch_size=16, classes=None,
                                                class_mode=None)

face_val = img_generator.flow_from_directory('data/val_images',target_size=(512,512),shuffle=False,
                                             seed=40,save_format='jpg',batch_size=4,classes=None,
                                             class_mode=None)

comic_val = img_generator.flow_from_directory('data/val_cartoon',target_size=(512,512),shuffle=False,
                                              seed=40,save_format='jpg',batch_size=4,classes=None,
                                              class_mode=None)

Found 8000 images belonging to 1 classes.
Found 8000 images belonging to 1 classes.
Found 2000 images belonging to 1 classes.
Found 2000 images belonging to 1 classes.


In [4]:
train_generator=zip(face_train, comic_train)
val_generator=zip(face_val, comic_val)

In [5]:

#########################################################################
model=Sequential(name="real2comic")
model.add(keras.Input(shape=(512,512,3)))
model.add(layers.Conv2D(3,3,activation='relu', padding='same'))
model.add(layers.Conv2D(32,3,activation='relu',strides=2, padding='same'))
model.add(layers.Conv2D(32,3,activation='relu',strides=2, padding='same'))
model.add(layers.Conv2D(32,3,activation='relu',strides=2, padding='same'))

model.add(layers.Conv2D(32,3,activation='relu',padding='same'))


model.add(layers.Conv2DTranspose(32,3,activation='relu', padding='same'))
model.add(layers.Conv2DTranspose(32,3,activation='relu', strides=2, padding='same'))
model.add(layers.Conv2DTranspose(32,3,activation='relu', strides=2, padding='same'))
model.add(layers.Conv2DTranspose(3,3, activation='sigmoid',strides=2, padding='same'))

model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])


#########################################################################

model.summary()

Model: "real2comic"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 3)       84        
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 32)      896       
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 conv2d_transpose (Conv2DTra  (None, 64, 64, 32)       9248      
 nspose)                                                         
                                                        

In [ ]:
model.fit(x=train_generator, validation_data = val_generator, steps_per_epoch=500,
          validation_steps=500, epochs=50)

Epoch 1/50
500/500 [==============================] - 467s 925ms/step - loss: 0.1488 - accuracy: 0.5467 - val_loss: 0.1283 - val_accuracy: 0.6522
Epoch 2/50
500/500 [==============================] - 460s 920ms/step - loss: 0.1278 - accuracy: 0.6954 - val_loss: 0.1239 - val_accuracy: 0.7231
Epoch 3/50
500/500 [==============================] - 457s 914ms/step - loss: 0.1243 - accuracy: 0.7133 - val_loss: 0.1225 - val_accuracy: 0.7029
Epoch 4/50
500/500 [==============================] - 457s 914ms/step - loss: 0.1229 - accuracy: 0.7202 - val_loss: 0.1211 - val_accuracy: 0.7165
Epoch 5/50
500/500 [==============================] - 457s 914ms/step - loss: 0.1215 - accuracy: 0.7244 - val_loss: 0.1200 - val_accuracy: 0.7277
Epoch 6/50
500/500 [==============================] - 457s 914ms/step - loss: 0.1210 - accuracy: 0.7272 - val_loss: 0.1195 - val_accuracy: 0.7455
Epoch 7/50
500/500 [==============================] - 457s 914ms/step - loss: 0.1201 - accuracy: 0.7314 - val_loss: 0.1191 -

In [ ]:
model.save("modelV5.h5")